In [1]:
from ipynb.fs.full.feature_engineering_final import X_Transformer_scaled
from ipynb.fs.full.feature_engineering_final import X_Transformer
from ipynb.fs.full.feature_engineering_final import DataLoader
from ipynb.fs.full.feature_engineering_final import TargetNormalizedRegressor
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNetCV
import xgboost as xgb
import numpy as np
import pandas as pd
from xgboost import plot_tree
from mlxtend.regressor import StackingCVRegressor
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import plot_tree
from matplotlib.pylab import rcParams
from sklearn.model_selection import KFold
from sklearn.svm import SVR

In [2]:
data_loader = DataLoader("train.csv","test.csv")

/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3429: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [3]:
X_train = data_loader.getX_train()
Y_train = data_loader.getY_train()
X_test = data_loader.getX_test()

In [4]:
x_transformer = X_Transformer_scaled()

In [5]:
kf = KFold(n_splits=10,shuffle = True,random_state = 0)

In [6]:
xgb = XGBRegressor(objective='reg:squarederror',eval_metric = 'rmse',learning_rate=0.01,n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)
svr = SVR(kernel="rbf",gamma=0.0003,C = 30,epsilon = 0.00008)

ridge = RidgeCV(alphas=[14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5])

lasso = LassoCV(cv=10)

elastic_net = ElasticNetCV(max_iter=1e7, alphas=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007],
                                        cv=kf, l1_ratio=[0.8, 0.85, 0.9, 0.95, 0.99, 1])

gbr = GradientBoostingRegressor(n_estimators=3700, learning_rate=0.01,
                                   max_depth=3, max_features='sqrt',
                                   loss='huber', random_state =42,subsample = 0.7)


scvr = TargetNormalizedRegressor(StackingCVRegressor(regressors=(xgb,svr,ridge,lasso,elastic_net,gbr),
                                meta_regressor=gbr,
                                use_features_in_secondary=True,cv=10,
                                  n_jobs = -1))

In [ ]:
learning_pipeline = Pipeline(steps=[('preprocess', x_transformer),('learn',scvr)])
learning_pipeline.fit(X_train,y=Y_train)





In [ ]:
#set up the parameters
#rcParams['figure.figsize'] = 100,90
#plot_tree(learning_pipeline.named_steps['learn'],num_trees=2799, rankdir='LR')#plt.show()

In [ ]:
Ypredicted = learning_pipeline.predict(X_test)

In [ ]:
resultDf = pd.DataFrame()
resultDf['Id'] = data_loader.get_Test_id()
resultDf['SalePrice'] = Ypredicted
resultDf
resultDf.to_csv('submission.csv',index = False)